In [1]:
import pandas as pd
import glob


In [2]:
data_path = "D:/unified_metor_content/fraud_detection/fraud_detection/" 

file_list = sorted(glob.glob(data_path + "*.pkl"))

# Read and concatenate all daily files
df_all = pd.concat([pd.read_pickle(file) for file in file_list], ignore_index=True)

In [3]:
df_all.head()


,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


In [4]:
df_all["fraud_amt"] = (df_all["TX_AMOUNT"] > 220).astype(int)


In [5]:
#df_all.head()

In [6]:
df_all['amt_sum'] = df_all['TERMINAL_ID'].map(df_all.groupby('TERMINAL_ID')['TX_AMOUNT'].sum())


In [7]:
#df_all

In [8]:
df_all['TX_DATETIME'] = pd.to_datetime(df_all['TX_DATETIME'])

In [9]:
df_all = df_all.sort_values(['TERMINAL_ID', 'TX_DATETIME'])

In [10]:
df_all['amt_14d'] = (
    df_all
    .groupby('TERMINAL_ID')
    .apply(lambda group: group.set_index('TX_DATETIME')
                           .TX_AMOUNT
                           .rolling('28D')
                           .sum()
                           .reset_index(drop=True))
    .reset_index(drop=True)
)

In [11]:
#df_all

In [12]:
df_all['TERMINAL_ID'] = df_all['TERMINAL_ID'].astype('category')

In [13]:
thresholds = df_all.groupby('TERMINAL_ID')['amt_14d'].quantile(0.95)

In [14]:


#thresholds_df = thresholds.reset_index()
#thresholds_df.columns = ['TERMINAL_ID', 'threshold_95']

# Merge back into the main DataFrame (fast and scalable)
#df_all = df_all.merge(thresholds_df, on='TERMINAL_ID', how='left')

In [15]:
#df_all['is_high_amt_7d'] = df_all['amt_7d'] > df_all['threshold_95']

In [16]:
#df_all

In [17]:
df_all['threshold_95'] = df_all['TERMINAL_ID'].map(thresholds)

# 3. Flag whether the 7-day sum exceeds that threshold
df_all['is_high_amt_14d'] = df_all['amt_14d'] > df_all['threshold_95']

In [18]:
#df_all

In [19]:
df_all['is_high_amt_14d'] = df_all['is_high_amt_14d'].astype(int) # 0 = not fraud, 1 = fraud flag
#df_all

In [20]:
pd.set_option('display.max_rows', None)

In [21]:
#df_all[['TX_FRAUD', 'is_high_amt_7d']]


In [22]:
matches = df_all['TX_FRAUD'] == df_all['is_high_amt_14d']
n_matches = matches.sum()
not_matches = (~matches).sum()
print(n_matches)
print(not_matches)

1649010
105145


In [23]:
#df_all[~matches]

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score

y_true = df_all['TX_FRAUD']
y_pred = df_all['is_high_amt_14d']

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
#so this feature is not good as standalone for predicting fraud

Precision: 0.0081
Recall: 0.0505
F1-score: 0.0139


In [25]:
y_true = df_all['TX_FRAUD']
y_pred = df_all['fraud_amt']

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 1.0000
Recall: 0.2219
F1-score: 0.3632
